# Starbucks Capstone Challenge

## Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

## Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

## Reading the Dataset and Preparing them for Analysis

In [1]:
# loading all libraries
import pandas as pd
import numpy as np
import math
import json

from utilities import *
from preprocess import *
%matplotlib inline

### Portfolio

In [10]:
portfolio = get_data('data/portfolio.json')
portfolio.shape

(10, 6)

In [16]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [17]:
df_po = process_portfolio(portfolio)
df_po.head()

,reward,difficulty,duration,offer_type,offer_id,ch_email,ch_mobile,ch_social,ch_web,tot_channels
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,3
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,4
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1,3
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1,3
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1,2


### Profile

In [11]:
profile = get_data('data/profile.json')
profile.shape

(17000, 5)

In [12]:
profile.head(5)

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [13]:
profile['id'].nunique()

17000

In [14]:
profile.isna().sum()

gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64

In [15]:
df_pf = process_profile(profile)
df_pf.head()

,age,customer_id,became_member_on,income,gn_F,gn_M,gn_O,member_for,pf_age_valid
0,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,0,0,0,4.47,False
1,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,1,0,0,4.06,True
2,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,0,0,0,3.06,False
3,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,1,0,0,4.24,True
4,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,0,0,0,4.00,False


### Transcript

In [2]:
transcript = get_data('data/transcript.json')
transcript.shape

(306534, 4)

In [3]:
transcript.sample(5)

,person,event,value,time
161331,825c2c2bb7aa4f29b1e882f40f0f8250,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},408
138532,1c1a3243eaed47058cc3167b282fd83a,offer completed,{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...,366
301038,23734b1dfcb94f9a8295153317a77c8c,offer viewed,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},690
151246,3909e7107fb7499ba95e0f2882f6b12d,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408
52320,7b249c93b5194a65911b9851729caa4b,transaction,{'amount': 16.31},162


In [4]:
transcript.isna().sum()

person    0
event     0
value     0
time      0
dtype: int64

In [5]:
transcript['person'].nunique()

17000

Transcript data contains 17000 users' 306534 transaction records.

In [6]:
transcript['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

Number of transactions for each customer varies and one customer's transaction looks like the following

In [7]:
transcript[transcript.person == '1e99a5d7cda04316ab07978b5fa641e4']

,person,event,value,time
32774,1e99a5d7cda04316ab07978b5fa641e4,transaction,{'amount': 13.4},60
54660,1e99a5d7cda04316ab07978b5fa641e4,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},168
66252,1e99a5d7cda04316ab07978b5fa641e4,offer viewed,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},168
112293,1e99a5d7cda04316ab07978b5fa641e4,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},336
127480,1e99a5d7cda04316ab07978b5fa641e4,offer viewed,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},342
203044,1e99a5d7cda04316ab07978b5fa641e4,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},504
214736,1e99a5d7cda04316ab07978b5fa641e4,offer viewed,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},504
258387,1e99a5d7cda04316ab07978b5fa641e4,transaction,{'amount': 21.71},576
258388,1e99a5d7cda04316ab07978b5fa641e4,offer completed,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7...,576
300796,1e99a5d7cda04316ab07978b5fa641e4,transaction,{'amount': 27.89},690


Same record is converted as follows for analysis purposes after preprocessing the dataset.

In [8]:
df_ts = process_transcript(transcript)
df_ts[df_ts.customer_id == '1e99a5d7cda04316ab07978b5fa641e4']

,customer_id,event,time,offer_id,amount,reward
32774,1e99a5d7cda04316ab07978b5fa641e4,transaction,60,NaN,13.40,NaN
54660,1e99a5d7cda04316ab07978b5fa641e4,offer received,168,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
66252,1e99a5d7cda04316ab07978b5fa641e4,offer viewed,168,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
112293,1e99a5d7cda04316ab07978b5fa641e4,offer received,336,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
127480,1e99a5d7cda04316ab07978b5fa641e4,offer viewed,342,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
203044,1e99a5d7cda04316ab07978b5fa641e4,offer received,504,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
214736,1e99a5d7cda04316ab07978b5fa641e4,offer viewed,504,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
258387,1e99a5d7cda04316ab07978b5fa641e4,transaction,576,NaN,21.71,NaN
258388,1e99a5d7cda04316ab07978b5fa641e4,offer completed,576,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,5.0
300796,1e99a5d7cda04316ab07978b5fa641e4,transaction,690,NaN,27.89,NaN


### Merge all datasets

In [18]:
trans_pf = pd.merge(df_ts, df_pf, on='customer_id', how="left")
df_tsall = pd.merge(trans_pf, df_po, on='offer_id', how='left')
df_tsall.shape

(306534, 23)

In [ ]:
# replacing all offer_ids with new name
df_transall =   df_tsall.replace({'offer_id' : offer_names})

### Personal Level Information